In [1]:
import numpy as np
import yt
import unyt
from yt import YTArray
from yt.data_objects.level_sets.api import Clump, find_clumps
import argparse
import os
from astropy.table import Table
from astropy.io import ascii
import multiprocessing as multi


import datetime
from scipy import interpolate
import shutil
import matplotlib.pyplot as plt
import cmasher as cmr
import matplotlib.colors as mcolors
import h5py
import trident

# These imports are FOGGIE-specific files
from foggie.utils.consistency import *
from foggie.utils.get_run_loc_etc import get_run_loc_etc
from foggie.utils.yt_fields import *
from foggie.utils.foggie_load import *
from foggie.utils.analysis_utils import *

# These imports for datashader plots
import datashader as dshader
from datashader.utils import export_image
import datashader.transfer_functions as tf
import pandas as pd
import matplotlib as mpl
import numpy as np
from yt.units.yt_array import YTQuantity
from scipy.ndimage import gaussian_filter

def generate_foggie_paths(halo, run, snap):
    # Define base paths
    foggie_base_dir = "/Users/vidasaeedzadeh/Projects/foggie_data/"
    code_base_path = "/Users/vidasaeedzadeh/Projects/foggie/foggie/"
    output_base_dir = "/Users/vidasaeedzadeh/Projects/foggie_outputs/"

    # Zero-pad the halo number to 6 digits
    halo_number = halo.zfill(6)

    # Define directory and file paths dynamically
    foggie_dir = os.path.join(foggie_base_dir, f"halo_{halo_number}", run + '/')
    snap_name = os.path.join(foggie_dir, snap, snap)
    halo_c_v_name = os.path.join(code_base_path, f"halo_infos/{halo_number}/{run}/halo_c_v")
    trackname = os.path.join(code_base_path, f"halo_tracks/{halo_number}/nref11n_selfshield_15/halo_track_200kpc_nref9")

    # Output directory (adjust based on needs)
    output_dir = output_base_dir

    # Return paths
    return foggie_dir,code_base_path, snap_name, halo_c_v_name, trackname, output_dir

# specify halo and snapshot
halo = '8508'
run = 'ludicrous/nref13c_nref9f.enhance'
snap = 'DD2509'

foggie_dir,code_path, snap_name, halo_c_v_name, trackname, output_dir = generate_foggie_paths(halo, run, snap)



# System and plotting settings
system = ''  # System you're using
plot = 'emission_FRB'  # Options: emission_map, emission_map_vbins, or emission_FRB or emission_FRB_binsmearing
ions = ['C IV', 'O VI']#['Lyalpha', 'Halpha', 'CIII','SiII','SiIII','SiIV','MgII']#['Lyalpha', 'Halpha', 'CIII', 'CIV', 'OVI','SiII','SiIII','SiIV','MgII']  
Dragonfly_limit = False
Aspera_limit = False
save_suffix = ""
file_suffix = ""


# Filtering settings (optional)
segmentation_filter='radial_velocity' # for categorizing inflow vs outflow it can also be 'metallicity'
filter_type = None  # Type of filter, e.g., 'temperature', 'density'
filter_value = None  # Value for the filter, e.g., 1e4 for temperature < 1e4 K

# Add Trident ion fields
def add_ion_fields(ds):
    trident.add_ion_fields(ds, ions=ions)
    return ds

def scale_by_metallicity(values,assumed_Z,wanted_Z):
    # The Cloudy calculations assumed a single metallicity (typically solar).
    # This function scales the emission by the metallicity of the gas itself to
    # account for this discrepancy.
    wanted_ratio = (10.**(wanted_Z))/(10.**(assumed_Z))
    return values*wanted_ratio

def make_Cloudy_table(table_index):
    # This function takes all of the Cloudy files and compiles them into one table
    # for use in the emission functions
    # table_index is the column in the Cloudy output files that is being read.
    # each table_index value corresponds to a different emission line

    # this is the the range and number of bins for which Cloudy was run
    # i.e. the temperature and hydrogen number densities gridded in the
    # Cloudy run. They must match or the table will be incorrect.
    hden_n_bins, hden_min, hden_max = 15, -5, 2 #17, -6, 2 #23, -9, 2
    T_n_bins, T_min, T_max = 51, 3, 8 #71, 2, 8

    hden=np.linspace(hden_min,hden_max,hden_n_bins)
    T=np.linspace(T_min,T_max, T_n_bins)
    table = np.zeros((hden_n_bins,T_n_bins))
    for i in range(hden_n_bins):
            table[i,:]=[float(l.split()[table_index]) for l in open(cloudy_path%(i+1)) if l[0] != "#"]
    return hden,T,table

def make_Cloudy_table_thin(table_index):
    hden_n_bins, hden_min, hden_max = 17, -5, 2
    T_n_bins, T_min, T_max = 51, 3, 8 #71, 2, 8

    hden=np.linspace(hden_min,hden_max,hden_n_bins)
    T=np.linspace(T_min,T_max, T_n_bins)
    table = np.zeros((hden_n_bins,T_n_bins))
    for i in range(hden_n_bins):
            table[i,:]=[float(l.split()[table_index]) for l in open(cloudy_path_thin%(i+1)) if l[0] != "#"]
    return hden,T,table

/Users/vidasaeedzadeh/miniforge3/envs/foggie/lib/python3.12/site-packages/dask/dataframe/__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
def _Emission_LyAlpha(field, data, unit_system='default'):
    H_N = np.log10(np.array(data["H_nuclei_density"]))
    Temperature = np.log10(np.array(data["Temperature"]))
    dia1 = bl_LA(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line = (10**dia1) * ((10.0**H_N)**2.0)
    
    if unit_system == 'default':
        emission_line = emission_line / (4. * np.pi * 1.63e-11)
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")


def _Emission_HAlpha(field, data, unit_system='default'):
    H_N = np.log10(np.array(data['H_nuclei_density']))
    Temperature = np.log10(np.array(data['Temperature']))
    dia1 = bl_HA(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line = (10.**dia1) * ((10.**H_N)**2.0)
    
    if unit_system == 'default':
        emission_line = emission_line / (4. * np.pi * 3.03e-12)
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")


def _Emission_CIII_977(field, data, unit_system='default'):
    H_N = np.log10(np.array(data["H_nuclei_density"]))
    Temperature = np.log10(np.array(data["Temperature"]))
    dia1 = bl_CIII_977(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line = (10.0**dia1) * ((10.0**H_N)**2.0)
    emission_line = scale_by_metallicity(emission_line, 0.0, np.log10(np.array(data['metallicity'])))
    
    if unit_system == 'default':
        emission_line = emission_line / (4. * np.pi * 2.03e-11)
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")

def _Emission_CIV_1548(field, data, unit_system='default'):
    H_N = np.log10(np.array(data["H_nuclei_density"]))
    Temperature = np.log10(np.array(data["Temperature"]))
    dia1 = bl_CIV_1(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line = (10.0**dia1) * ((10.0**H_N)**2.0)
    emission_line = scale_by_metallicity(emission_line, 0.0, np.log10(np.array(data['metallicity'])))
    
    if unit_system == 'default':
        emission_line = emission_line / (4. * np.pi * 1.28e-11)
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")


def _Emission_OVI(field, data, unit_system='default'):
    H_N = np.log10(np.array(data["H_nuclei_density"]))
    Temperature = np.log10(np.array(data["Temperature"]))
    dia1 = bl_OVI_1(H_N, Temperature)
    dia2 = bl_OVI_2(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    dia2[idx] = -200.
    emission_line = ((10.0**dia1) + (10**dia2)) * ((10.0**H_N)**2.0)
    emission_line = scale_by_metallicity(emission_line, 0.0, np.log10(np.array(data['metallicity'])))
    
    if unit_system == 'default':
        emission_line = emission_line / (4. * np.pi * 1.92e-11)
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")


def _Emission_SiIII_1207(field, data, unit_system='default'):
    H_N = np.log10(np.array(data["H_nuclei_density"]))
    Temperature = np.log10(np.array(data["Temperature"]))
    dia1 = bl_SiIII_1207(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line = (10.0**dia1) * ((10.0**H_N)**2.0)
    emission_line = scale_by_metallicity(emission_line, 0.0, np.log10(np.array(data['metallicity'])))
    
    if unit_system == 'default':
        emission_line = emission_line / (4. * np.pi * 1.65e-11)
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")


def _Emission_SiII_1814(field, data, unit_system='default'):
    H_N = np.log10(np.array(data["H_nuclei_density"]))
    Temperature = np.log10(np.array(data["Temperature"]))
    dia1 = bl_SiIII_1207(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line = (10.0**dia1) * ((10.0**H_N)**2.0)
    emission_line = scale_by_metallicity(emission_line, 0.0, np.log10(np.array(data['metallicity'])))
    
    if unit_system == 'default':
        emission_line = emission_line / (4.*np.pi*1.65e-11)
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")

def _Emission_SiIV_1394(field, data, unit_system='default'):
    H_N = np.log10(np.array(data["H_nuclei_density"]))
    Temperature = np.log10(np.array(data["Temperature"]))
    dia1 = bl_SiIII_1207(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line = (10.0**dia1) * ((10.0**H_N)**2.0)
    emission_line = scale_by_metallicity(emission_line, 0.0, np.log10(np.array(data['metallicity'])))
    
    if unit_system == 'default':
        emission_line = emission_line / (4.*np.pi*1.65e-11)
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")

def _Emission_MgII_2796(field, data, unit_system='default'):
    H_N = np.log10(np.array(data["H_nuclei_density"]))
    Temperature = np.log10(np.array(data["Temperature"]))
    dia1 = bl_SiIII_1207(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line = (10.0**dia1) * ((10.0**H_N)**2.0)
    emission_line = scale_by_metallicity(emission_line, 0.0, np.log10(np.array(data['metallicity'])))
    
    if unit_system == 'default':
        emission_line = emission_line / (4.*np.pi*1.65e-11) # what should be instead of 1.65e-11 for MgII? or anyother new element I use?
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")


In [3]:
def filter_ds(box):
    '''This function filters the yt data object passed in as 'box' into inflow and outflow regions,
    based on metallicity, and returns the box filtered into these regions.'''

    if (segmentation_filter=='metallicity'):
        box_#inflow = box.include_below(('gas','metallicity'), 0.01, 'Zsun')
        box_outflow = box.include_above(('gas','metallicity'), 1., 'Zsun')
        box_neither = box.include_above(('gas','metallicity'), 0.01, 'Zsun')
        box_neither = box_neither.include_below(('gas','metallicity'), 1., 'Zsun')
    elif (segmentation_filter=='radial_velocity'):
        box_inflow = box.include_below(('gas','radial_velocity_corrected'), -100., 'km/s')
        box_outflow = box.include_above(('gas','radial_velocity_corrected'), 200., 'km/s')
        box_neither = box.include_above(('gas','radial_velocity_corrected'), -100., 'km/s')
        box_neither = box_neither.include_below(('gas','radial_velocity_corrected'), 200., 'km/s')

    return box_inflow, box_outflow, box_neither

def make_FRB(ds, refine_box, snap, ions, unit_system='default', filter_type=None, filter_value=None,resolution=100):
    '''This function takes the dataset 'ds' and the refine box region 'refine_box' and
    makes a fixed resolution buffer of surface brightness from edge-on, face-on,
    and arbitrary orientation projections of all ions in the list 'ions'.'''

    halo_name = halo_dict[str(halo)]

    # Determine the resolution based on pixel size
    pix_res = float(np.min(refine_box['dx'].in_units('kpc'))) #pix_res represents the physical size of a single grid cell in the refined box.
    #res = int(ds.refine_width / pix_res)
    #res = int(100 / pix_res)
    res = int(resolution / pix_res) # to be able to run using different resolution

    #fov and bin size
    # Ensure fov_kpc is in kpc
    if not hasattr(ds.refine_width, 'in_units'):
        fov_kpc = YTQuantity(ds.refine_width, 'kpc')  # Wrap in YTQuantity with units
    else:
        fov_kpc = ds.refine_width.in_units('kpc')  # Convert to kpc if it has units
    
    # Convert to numeric value (without units) for calculations
    fov_kpc_value = fov_kpc.v
    
    # Calculate bin size
    bin_size_kpc = round(fov_kpc_value / res,1)

    #for my understanding:
    print('pix_res',pix_res)
    print('res',res)
    print('fov_kpc_value',fov_kpc_value)
    print('bin_size_kpc',bin_size_kpc)
    
    print('z=%.1f' % ds.get_parameter('CosmologyCurrentRedshift', 1))

    # Apply inflow/outflow or disk/CGM filtering using the filter_ds function if specified
    if filter_type == 'inflow_outflow':
        # Apply the inflow/outflow filtering
        box_inflow, box_outflow, box_neither = filter_ds(ds.all_data())
        data_sources = {'inflow': box_inflow, 'outflow': box_outflow, 'neither': box_neither}

    elif filter_type == 'disk_cgm':
        # Apply the disk/CGM filtering
        box_disk, box_cgm = filter_ds_disk_cgm(ds)
        data_sources = {'disk': box_disk, 'cgm': box_cgm}
        
    else:
        # Standard filtering or no filter
        data_sources = {'all': ds.all_data()}
        if filter_type and filter_value:
            if filter_type == 'temperature':
                data_sources['all'] = data_sources['all'].cut_region([f"(obj['gas', 'temperature'] < {filter_value})"])
            elif filter_type == 'density':
                data_sources['all'] = data_sources['all'].cut_region([f"(obj['gas', 'density'] > {filter_value})"])
            else:
                raise ValueError("Unsupported filter type. Supported types: 'temperature', 'density'.")

    # Define the unit string based on unit_system
    if unit_system == 'default':
        unit_label = '[photons s$^{-1}$ cm$^{-2}$ sr$^{-1}$]'
    elif unit_system == 'ALT':
        unit_label = '[erg s$^{-1}$ cm$^{-2}$ arcsec$^{-2}$]'
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")

    # Create HDF5 file for saving emission maps
    save_path = prefix + f'FRBs/res_{bin_size_kpc}/' 
    os.makedirs(save_path, exist_ok=True)  # Ensure the directory exists
    f = h5py.File(save_path + halo_name + '_emission_maps' + save_suffix + '.hdf5', 'a')
    grp = f.create_group('z=%.1f' % ds.get_parameter('CosmologyCurrentRedshift', 1))
    grp.attrs.create("image_extent_kpc", ds.refine_width)
    grp.attrs.create("redshift", ds.get_parameter('CosmologyCurrentRedshift'))
    grp.attrs.create("halo_name", halo_name)
    grp.attrs.create("emission_units", unit_label)
    grp.attrs.create("gas_density_units", 'g/cm^2')
    grp.attrs.create("stars_density_units", 'Msun/kpc^2')
    grp.attrs.create("bin_size_kpc", bin_size_kpc)
    grp.attrs.create("number_of_bins", res)





    # Loop through ions and create projections for each region
    for region, data_source in data_sources.items():
        for ion in ions:
            print(ion)

            #Edge-on projection
            proj_edge = yt.ProjectionPlot(ds, ds.x_unit_disk, ('gas', 'Emission_' + ions_dict[ion]),
                                          center=ds.halo_center_kpc, data_source=data_source,width=(ds.refine_width, 'kpc'),
                                          north_vector=ds.z_unit_disk, buff_size=[res, res], method = 'integrate', weight_field=None)
            frb_edge = proj_edge.frb[('gas', 'Emission_' + ions_dict[ion])]
            dset1 = grp.create_dataset(f"{ion}_emission_edge_{region}", data=frb_edge)

            # Set colormap and save projection plot
            mymap = cmr.get_sub_cmap('cmr.flamingo', 0.2, 0.8)
            mymap.set_bad("#421D0F")
            proj_edge.set_cmap('Emission_' + ions_dict[ion], mymap)
            proj_edge.set_zlim('Emission_' + ions_dict[ion], zlim_dict[ion][0], zlim_dict[ion][1])
            proj_edge.set_colorbar_label('Emission_' + ions_dict[ion], label_dict[ion] + 'Emission' + unit_label)
            proj_edge.set_font_size(20)
            proj_edge.annotate_timestamp(corner='upper_left', redshift=True, time=True, draw_inset_box=True)
            proj_edge.save(save_path + f'{snap}_{ion}_emission_map_edge-on_{region}' + save_suffix + '.png')

            # Face-on projection
            proj_face = yt.ProjectionPlot(ds, ds.z_unit_disk, ('gas', 'Emission_' + ions_dict[ion]),
                                          center=ds.halo_center_kpc, data_source=data_source,width=(ds.refine_width, 'kpc'),
                                          north_vector=ds.x_unit_disk, buff_size=[res, res],weight_field=None)
            frb_face = proj_face.frb[('gas', 'Emission_' + ions_dict[ion])]
            dset2 = grp.create_dataset(f"{ion}_emission_face_{region}", data=frb_face)

            # Set colormap and save projection plot
            proj_face.set_cmap('Emission_' + ions_dict[ion], mymap)
            proj_face.set_zlim('Emission_' + ions_dict[ion], zlim_dict[ion][0], zlim_dict[ion][1])
            proj_face.set_colorbar_label('Emission_' + ions_dict[ion], label_dict[ion] + 'Emission' + unit_label)
            proj_face.set_font_size(20)
            proj_face.annotate_timestamp(corner='upper_left', redshift=True, time=True, draw_inset_box=True)
            proj_face.save(save_path + f'{snap}_{ion}_emission_map_face-on_{region}' + save_suffix + '.png')

    # Close the HDF5 file after saving the datasets
    print('finished')
    f.close()

def emission_map_vbins(ds, snap, ions,unit_system='default', filter_type=None, filter_value=None):
    '''Makes many emission maps for each ion in 'ions', oriented both edge-on and face-on, for each line-of-sight velocity bin.'''

    vbins = np.arange(-500., 550., 50.)  # Velocity bins
    ad = ds.all_data()

    for i in range(len(ions)):
        ion = ions[i]

        # Choose colormap based on ion and emission limits
        if (ion == 'Halpha') and Dragonfly_limit:
            cmap1 = cmr.take_cmap_colors('cmr.flamingo', 9, cmap_range=(0.4, 0.8), return_fmt='rgba')
            cmap2 = cmr.take_cmap_colors('cmr.neutral_r', 3, cmap_range=(0.2, 0.6), return_fmt='rgba')
            cmap = np.hstack([cmap2, cmap1])
            mymap = mcolors.LinearSegmentedColormap.from_list('cmap', cmap)
        elif (ion == 'OVI') and Aspera_limit:
            cmap1 = cmr.take_cmap_colors('cmr.flamingo', 4, cmap_range=(0.4, 0.8), return_fmt='rgba')
            cmap2 = cmr.take_cmap_colors('cmr.neutral_r', 6, cmap_range=(0.2, 0.6), return_fmt='rgba')
            cmap = np.hstack([cmap2, cmap1])
            mymap = mcolors.LinearSegmentedColormap.from_list('cmap', cmap)
        else:
            mymap = cmr.get_sub_cmap('cmr.flamingo', 0.2, 0.8)
        mymap.set_bad(mymap.colors[0])

        # Loop through each velocity bin
        for v in range(len(vbins) - 1):
            # Filter the data by the current velocity bin
            vbox = ds.cut_region(ad, [f"obj[('gas', 'vx_disk')] > {vbins[v]:.1f}"])
            vbox = ds.cut_region(vbox, [f"obj[('gas', 'vx_disk')] < {vbins[v+1]:.1f}"])

            # Apply filtering if specified (e.g., temperature or density cut)
            if filter_type and filter_value:
                if filter_type == 'temperature':
                    vbox = vbox.cut_region([f"(obj['gas', 'temperature'] < {filter_value})"])
                elif filter_type == 'density':
                    vbox = vbox.cut_region([f"(obj['gas', 'density'] > {filter_value})"])
                else:
                    raise ValueError("Unsupported filter type. Supported types: 'temperature', 'density'.")

            # Edge-on projection
            proj_edge = yt.ProjectionPlot(ds, ds.x_unit_disk, ('gas', 'Emission_' + ions_dict[ion]), 
                                          center=ds.halo_center_kpc, width=(ds.refine_width, 'kpc'),
                                          north_vector=ds.z_unit_disk, data_source=vbox)
            proj_edge.set_cmap('Emission_' + ions_dict[ion], mymap)
            proj_edge.set_zlim('Emission_' + ions_dict[ion], zlim_dict[ion][0], zlim_dict[ion][1])
            proj_edge.set_colorbar_label('Emission_' + ions_dict[ion], label_dict[ion] + 'Emission' + unit_label)
            proj_edge.set_font_size(20)
            proj_edge.annotate_title(f'$%d < v_{{\\rm los}} < %d$' % (vbins[v], vbins[v+1]))
            proj_edge.annotate_timestamp(corner='upper_left', redshift=True, time=True, draw_inset_box=True)
            proj_edge.save(prefix + 'EmissionMap/' + snap + '_' + ion + '_emission_map_edge-on_vbin' + str(v) + save_suffix + '.png')

            # Face-on projection
            proj_face = yt.ProjectionPlot(ds, ds.z_unit_disk, ('gas', 'Emission_' + ions_dict[ion]), 
                                          center=ds.halo_center_kpc, width=(ds.refine_width, 'kpc'),
                                          north_vector=ds.x_unit_disk, data_source=vbox)
            proj_face.set_cmap('Emission_' + ions_dict[ion], mymap)
            proj_face.set_zlim('Emission_' + ions_dict[ion], zlim_dict[ion][0], zlim_dict[ion][1])
            proj_face.set_colorbar_label('Emission_' + ions_dict[ion], label_dict[ion] + 'Emission' + unit_label)
            proj_face.set_font_size(20)
            proj_face.annotate_title(f'$%d < v_{{\\rm los}} < %d$' % (vbins[v], vbins[v+1]))
            proj_face.annotate_timestamp(corner='upper_left', redshift=True, time=True, draw_inset_box=True)
            proj_face.save(prefix + 'EmissionMap/' + snap + '_' + ion + '_emission_map_face-on_vbin' + str(v) + save_suffix + '.png')


def make_mass_FRB(ds, refine_box, snap, ions, filter_type=None, filter_value=None, resolution=100):
    '''This function calculates and saves mass FRBs and total mass for each ion.'''

    halo_name = halo_dict[str(halo)]

    # Determine resolution and bin size
    pix_res = float(np.min(refine_box['dx'].in_units('kpc')))
    res = int(resolution / pix_res)

    # Ensure fov_kpc is in kpc
    if not hasattr(ds.refine_width, 'in_units'):
        fov_kpc = YTQuantity(ds.refine_width, 'kpc')  # Wrap in YTQuantity with units
    else:
        fov_kpc = ds.refine_width.in_units('kpc')  # Convert to kpc if it has units
    
    # Convert to numeric value (without units) for calculations
    fov_kpc_value = fov_kpc.v

    # Calculate bin size
    bin_size_kpc = round(fov_kpc_value / res, 1)

    print('pix_res:', pix_res)
    print('res:', res)
    print('fov_kpc:', fov_kpc)
    print('bin_size_kpc:', bin_size_kpc)

    # Apply filtering (if specified)
    if filter_type == 'inflow_outflow':
        box_inflow, box_outflow, box_neither = filter_ds(ds.all_data())
        data_sources = {'inflow': box_inflow, 'outflow': box_outflow, 'neither': box_neither}
    elif filter_type == 'disk_cgm':
        box_disk, box_cgm = filter_ds_disk_cgm(ds)
        data_sources = {'disk': box_disk, 'cgm': box_cgm}
    else:
        data_sources = {'all': ds.all_data()}
        if filter_type and filter_value:
            if filter_type == 'temperature':
                data_sources['all'] = data_sources['all'].cut_region([f"(obj['gas', 'temperature'] < {filter_value})"])
            elif filter_type == 'density':
                data_sources['all'] = data_sources['all'].cut_region([f"(obj['gas', 'density'] > {filter_value})"])
            else:
                raise ValueError("Unsupported filter type. Supported types: 'temperature', 'density'.")

    # Create HDF5 file for saving mass maps
    save_path = prefix + f'FRBs/res_{bin_size_kpc}/' 
    os.makedirs(save_path, exist_ok=True)  # Ensure the directory exists
    f = h5py.File(save_path + halo_name + '_emission_maps' + save_suffix + '.hdf5', 'a')
    # Check if the group already exists
    redshift_group_name = 'z=%.1f' % ds.get_parameter('CosmologyCurrentRedshift', 1)
    if redshift_group_name not in f:
        grp = f.create_group(redshift_group_name)
        grp.attrs.create("image_extent_kpc", ds.refine_width)
        grp.attrs.create("redshift", ds.get_parameter('CosmologyCurrentRedshift'))
        grp.attrs.create("halo_name", halo_name)
        grp.attrs.create("bin_size_kpc", bin_size_kpc)
        grp.attrs.create("number_of_bins", res)
    else:
        grp = f[redshift_group_name]  # Open the existing group

    # Compute pixel area in cm^2
    pixel_area_kpc2 = (fov_kpc / res) ** 2  # Pixel area in kpc^2
    pixel_area_cm2 = pixel_area_kpc2.in_units('cm**2')  # Convert to cm^2

    # Loop through ions and create projections for each region
    for region, data_source in data_sources.items():
        for ion in ions:
            print(f"Processing ion: {ion}")

            # Replace mass field with ion density
            density_field = ('gas', ions_density_dict[ion]) 

            # Edge-on projection (surface density)
            proj_edge = yt.ProjectionPlot(ds, ds.x_unit_disk, density_field,
                                          center=ds.halo_center_kpc, data_source=data_source,
                                          width=(ds.refine_width, 'kpc'), north_vector=ds.z_unit_disk,
                                          buff_size=[res, res], weight_field=None)
            frb_edge = proj_edge.frb[density_field]  # Surface density in g/cm^2
            frb_edge_mass = (frb_edge * pixel_area_cm2).in_units('Msun') 
            # Compute total mass for edge-on projection
            total_mass_edge = (frb_edge * pixel_area_cm2).sum().in_units('Msun')  # Convert to solar masses

            # Save mass frb and total mass in HDF5
            dset1 = grp.create_dataset(f"{ion}_mass_edge_{region}", data=frb_edge_mass)
            dset1.attrs.create("total_mass_Msun", total_mass_edge)

            # Face-on projection (surface density)
            proj_face = yt.ProjectionPlot(ds, ds.z_unit_disk, density_field,
                                          center=ds.halo_center_kpc, data_source=data_source,
                                          width=(ds.refine_width, 'kpc'), north_vector=ds.x_unit_disk,
                                          buff_size=[res, res], weight_field=None)
            frb_face = proj_face.frb[density_field]  # Surface density in g/cm^2
            frb_face_mass = (frb_face* pixel_area_cm2).in_units('Msun') 
            # Compute total mass for face-on projection
            total_mass_face = (frb_face * pixel_area_cm2).sum().in_units('Msun')  # Convert to solar masses

            # Save surface density and total mass in HDF5
            dset2 = grp.create_dataset(f"{ion}_mass_face_{region}", data=frb_face_mass)
            dset2.attrs.create("total_mass_Msun", total_mass_face)

            print(f"Edge total mass for {ion}: {total_mass_edge}")
            print(f"Face total mass for {ion}: {total_mass_face}")

    # Close the HDF5 file after saving the datasets
    print('Mass FRBs finished')
    f.close()


def load_and_calculate(snap, ions, unit_system='default', filter_type=None, filter_value=None, resolution=100):

    '''Loads the simulation snapshot and makes the requested plots, with optional filtering.'''

    # Load simulation output
    if system == 'pleiades_cassi':
        print('Copying directory to /tmp')
        snap_dir = '/nobackup/clochhaa/tmp/' + halo + '/' + run + '/' + target_dir + '/' + snap
        os.makedirs(snap_dir)
        snap_name = foggie_dir + run_dir + snap + '/' + snap
    else:
        snap_name = foggie_dir + snap + '/' + snap
    
    ds, refine_box = foggie_load(snap_name, trackname, do_filter_particles=True, halo_c_v_name=halo_c_v_name, disk_relative=True, correct_bulk_velocity=True)#, smooth_AM_name=smooth_AM_name)
    zsnap = ds.get_parameter('CosmologyCurrentRedshift')
    add_ion_fields(ds)

    # Generate emission maps based on the plot type
    if 'emission_map' in plot:
        if 'vbins' not in plot:
            emission_map(ds, snap, ions, filter_type=filter_type, filter_value=filter_value)
        else:
            # Call velocity-binned emission map function with optional filtering
            emission_map_vbins(ds, snap, ions, unit_system=unit_system, filter_type=filter_type, filter_value=filter_value)
    
    if 'emission_FRB' in plot:
        make_FRB(ds, refine_box, snap, ions, unit_system=unit_system, filter_type=filter_type, filter_value=filter_value, resolution=resolution)
        make_mass_FRB(ds, refine_box, snap, ions, filter_type=filter_type, filter_value=filter_value, resolution=resolution)

        

    

if __name__ == "__main__":


    # if ('feedback' in run) and ('track' in run):
    #     foggie_dir = '/nobackup/jtumlins/halo_008508/feedback-track/'
    #     run_dir = run + '/'
    
    # Set directory for output location, making it if necessary
    prefix = output_dir + 'ions_halo_00' + halo + '/' + run + '/'
    if not (os.path.exists(prefix)): os.system('mkdir -p ' + prefix)
    table_loc = prefix + 'Tables/'

    print('foggie_dir: ', foggie_dir)
    catalog_dir = code_path + 'halo_infos/00' + halo + '/' + run + '/'
    halo_c_v_name = catalog_dir + 'halo_c_v'
    #smooth_AM_name = catalog_dir + 'AM_direction_smoothed'

    cloudy_path = code_path + "emission/cloudy_z0_selfshield/sh_z0_HM12_run%i.dat"
    # These are the typical units that Lauren uses
    # NOTE: This is a volumetric unit since it's for the emissivity of each cell
    # Emission / surface brightness comes from the projections
    emission_units = 's**-1 * cm**-3 * steradian**-1'
    ytEmU = unyt.second**-1 * unyt.cm**-3 * unyt.steradian**-1

    # These are a second set of units that a lot of observers prefer
    # NOTE: This is a volumetric unit since it's for the emissivity of each cell
    # Emission / surface brightness comes from the projections
    emission_units_ALT = 'erg * s**-1 * cm**-3 * arcsec**-2'
    ytEmUALT = unyt.erg * unyt.second**-1 * unyt.cm**-3 * unyt.arcsec**-2

    ####################################
    ## BEGIN CREATING EMISSION FIELDS ##
    ####################################

    # To make the emissivity fields, you need to follow a number of steps
    # 1. Read in the Cloudy values for a given emission line
    # 2. Create the n_H and T grids that represent the desired range of values
    # 3. Set up interpolation function for the emissivity values across the grids
    #    so the code can use the n_H and T values of a simulation grid cell to
    #    interpolate the correct emissivity value
    # 4. Define the emission field for the line
    # 5. Add the line as a value in yt

    ############################
    # Function to register emission fields with unit options
    def register_emission_field_with_unit(field_name, function, emission_units, unit_system):
        yt.add_field(
            ('gas', field_name),
            units=emission_units if unit_system == 'default' else emission_units_ALT,
            function=lambda field, data: function(field, data, unit_system=unit_system),
            take_log=True,
            force_override=True,
            sampling_type='cell',
        )
    
    ############################
    # Unit system setting (can be passed dynamically)
    unit_system = 'ALT'  # Change this to 'ALT' as needed
    
    ############################
    # H-Alpha
    hden_pts, T_pts, table_HA = make_Cloudy_table(2)
    hden_pts, T_pts = np.meshgrid(hden_pts, T_pts)
    pts = np.array((hden_pts.ravel(), T_pts.ravel())).T
    
    sr_HA = table_HA.T.ravel()
    bl_HA = interpolate.LinearNDInterpolator(pts, sr_HA)
    register_emission_field_with_unit('Emission_HAlpha', _Emission_HAlpha, emission_units, unit_system)
    
    ############################
    # Ly-Alpha
    hden_pts, T_pts, table_LA = make_Cloudy_table(1)
    sr_LA = table_LA.T.ravel()
    bl_LA = interpolate.LinearNDInterpolator(pts, sr_LA)
    register_emission_field_with_unit('Emission_LyAlpha', _Emission_LyAlpha, emission_units, unit_system)
    
    ############################
    # CIII 977
    hden_pts, T_pts, table_CIII_977 = make_Cloudy_table(7)
    sr_CIII_977 = table_CIII_977.T.ravel()
    bl_CIII_977 = interpolate.LinearNDInterpolator(pts, sr_CIII_977)
    register_emission_field_with_unit('Emission_CIII_977', _Emission_CIII_977, emission_units, unit_system)
    
    ############################
    # CIV 1548
    hden_pts, T_pts, table_CIV_1 = make_Cloudy_table(3)
    sr_CIV_1 = table_CIV_1.T.ravel()
    bl_CIV_1 = interpolate.LinearNDInterpolator(pts, sr_CIV_1)
    register_emission_field_with_unit('Emission_CIV_1548', _Emission_CIV_1548, emission_units, unit_system)
    
    ############################
    # O VI (1032 and 1037 combined)
    hden_pts, T_pts, table_OVI_1 = make_Cloudy_table(5)
    hden_pts, T_pts, table_OVI_2 = make_Cloudy_table(6)
    sr_OVI_1 = table_OVI_1.T.ravel()
    sr_OVI_2 = table_OVI_2.T.ravel()
    bl_OVI_1 = interpolate.LinearNDInterpolator(pts, sr_OVI_1)
    bl_OVI_2 = interpolate.LinearNDInterpolator(pts, sr_OVI_2)
    register_emission_field_with_unit('Emission_OVI', _Emission_OVI, emission_units, unit_system)
    
    ############################
    # SiIII 1207
    cloudy_path_thin = code_path + "emission/cloudy_z0_HM05/bertone_run%i.dat"
    hden_pts, T_pts, table_SiIII_1207 = make_Cloudy_table_thin(11)
    hden_pts, T_pts = np.meshgrid(hden_pts, T_pts)
    pts = np.array((hden_pts.ravel(), T_pts.ravel())).T
    sr_SiIII_1207 = table_SiIII_1207.T.ravel()
    bl_SiIII_1207 = interpolate.LinearNDInterpolator(pts, sr_SiIII_1207)
    register_emission_field_with_unit('Emission_SiIII_1207', _Emission_SiIII_1207, emission_units, unit_system)


    ############################
    ions_dict = {'Lyalpha':'LyAlpha', 'Halpha':'HAlpha', 'C III':'CIII_977',
                 'C IV':'CIV_1548','O VI':'OVI', 'Si III':'SiIII_1207'}
    ions_density_dict = {'Lyalpha':'LyAlpha', 'Halpha':'HAlpha', 'C III':'C_p2_density',
                 'C IV':'C_p3_density','O VI':'O_p5_density', 'Si III':'Si_p2_density', 'Si II':'Si_p1_density', 'Si IV':'Si_p3_density','Mg II':'Mg_p1_density'}
    
    label_dict = {'Lyalpha':r'Ly-$\alpha$', 'Halpha':r'H$\alpha$', 'C III':'C III',
                'C IV':'C IV','O VI':'O VI', 'Si III':'Si III'}

    if unit_system  == 'default':
        zlim_dict = {'Lyalpha':[1e-1,1e7], 'Halpha':[1e-1,1e6], 'C III':[1e-4,1e0],
                 'C IV':[1e-2,1e4], 'O VI':[1e-2,1e5], 'Si III':[1e-1,1e4]}
    elif unit_system == 'ALT':
        zlim_dict = {'Lyalpha':[1e-22,1e-16], 'Halpha':[1e-22,1e-16], 'C III':[1e-26,1e-16],
                 'C IV':[1e-23,1e-16], 'O VI':[1e-23,1e-16], 'Si III':[1e-22,1e16]}
        



resolution_list = [100,50,10,3.25]
for reso in resolution_list:
    load_and_calculate(snap, ions, unit_system='ALT', resolution=reso) #or choose 'ALT' for erg unit
    

yt : [INFO     ] 2024-11-27 16:51:49,337 Parameters: current_time              = 638.06651531954
yt : [INFO     ] 2024-11-27 16:51:49,337 Parameters: domain_dimensions         = [256 256 256]
yt : [INFO     ] 2024-11-27 16:51:49,338 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2024-11-27 16:51:49,338 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2024-11-27 16:51:49,338 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2024-11-27 16:51:49,338 Parameters: current_redshift          = 0.0021037994044062
yt : [INFO     ] 2024-11-27 16:51:49,338 Parameters: omega_lambda              = 0.715
yt : [INFO     ] 2024-11-27 16:51:49,339 Parameters: omega_matter              = 0.285
yt : [INFO     ] 2024-11-27 16:51:49,339 Parameters: omega_radiation           = 0
yt : [INFO     ] 2024-11-27 16:51:49,339 Parameters: hubble_constant           = 0.695


foggie_dir:  /Users/vidasaeedzadeh/Projects/foggie_data/halo_008508/ludicrous/nref13c_nref9f.enhance/
Opening snapshot /Users/vidasaeedzadeh/Projects/foggie_data/halo_008508/ludicrous/nref13c_nref9f.enhance/DD2509/DD2509
get_refine_box: using this location:        col1          col2     col3     col4     col5     col6     col7   col8
------------------ -------- -------- -------- -------- -------- -------- ----
0.0021111231691204 0.488873 0.470349 0.508524 0.490873 0.472349 0.510524    9


Parsing Hierarchy : 100%|████████████████████████████████████████████████████████████████████████████████| 14472/14472 [00:00<00:00, 27131.86it/s]
yt : [INFO     ] 2024-11-27 16:51:50,047 Gathering a field list (this may take a moment.)


This halo_c_v file doesn't exist, calculating halo center...
get_halo_center: code_length code_velocity
get_halo_center: obtained the spherical region
get_halo_center: extracted the DM density
get_halo_center: we have obtained the preliminary center
got the velocities
get_halo_center: located the main halo at: [0.48985743522644043, 0.47122740745544434, 0.5095312595367432] [unyt_quantity(0.001008, 'code_velocity'), unyt_quantity(-0.00177564, 'code_velocity'), unyt_quantity(0.0011421, 'code_velocity')]
filtering young_stars particles...
filtering young_stars3 particles...
filtering young_stars8 particles...
filtering old_stars particles...
filtering stars particles...
filtering dm particles...
using particle type  young_stars  to derive angular momentum
found angular momentum vector


yt : [INFO     ] 2024-11-27 16:52:09,968 xlim = -0.001000 0.001000
yt : [INFO     ] 2024-11-27 16:52:09,968 ylim = -0.001000 0.001000
yt : [INFO     ] 2024-11-27 16:52:09,968 zlim = -0.500000 0.500000
yt : [INFO     ] 2024-11-27 16:52:09,969 Making a fixed resolution buffer of (('gas', 'Emission_CIV_1548')) 1460 by 1460


...

finished
pix_res: 0.06846562510948173
res: 47
fov_kpc: 287.1656452591999 kpc
bin_size_kpc: 6.1
Processing ion: C IV


yt : [INFO     ] 2024-11-27 18:30:43,271 xlim = -0.001000 0.001000
yt : [INFO     ] 2024-11-27 18:30:43,271 ylim = -0.001000 0.001000
yt : [INFO     ] 2024-11-27 18:30:43,271 zlim = -0.500000 0.500000
yt : [INFO     ] 2024-11-27 18:30:43,272 Making a fixed resolution buffer of (('gas', 'C_p3_density')) 47 by 47
yt : [INFO     ] 2024-11-27 18:33:20,502 xlim = -0.001000 0.001000
yt : [INFO     ] 2024-11-27 18:33:20,503 ylim = -0.001000 0.001000
yt : [INFO     ] 2024-11-27 18:33:20,503 zlim = -0.500000 0.500000
yt : [INFO     ] 2024-11-27 18:33:20,503 Making a fixed resolution buffer of (('gas', 'O_p5_density')) 47 by 47


Edge total mass for C IV: 14889.77031650094 Msun
Face total mass for C IV: 13020.844731656787 Msun
Processing ion: O VI


yt : [INFO     ] 2024-11-27 18:35:57,615 xlim = -0.001000 0.001000
yt : [INFO     ] 2024-11-27 18:35:57,616 ylim = -0.001000 0.001000
yt : [INFO     ] 2024-11-27 18:35:57,616 zlim = -0.500000 0.500000
yt : [INFO     ] 2024-11-27 18:35:57,616 Making a fixed resolution buffer of (('gas', 'O_p5_density')) 47 by 47


Edge total mass for O VI: 140599.88012825945 Msun
Face total mass for O VI: 143299.7302445669 Msun
Mass FRBs finished
